In [1]:
!pip install sdv
!pip3 install azure-storage-blob azure-identity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.8/169.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 42.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of sdmetrics to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 kB 24.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_

In [2]:
from sdv.single_table import CTGANSynthesizer
import numpy as np
from sdv.sampling import Condition
import pandas as pd
from datetime import datetime

In [7]:

def synthetic_data_gen(no_of_records=100,type="Both",percentage="50%"):
    """
    synthatic_data_gen used to generate the synthatic data

    parameters:
    no_of_records: int , minimum value is 1
                   no_of_records will tell to the model how many records need to be generated
    type:str ["Fraud","No Fraud","Both"]
              type is tell to model what type of data need to be generated
    Percentage:str  tell to the model how mch fruad data  need to be generated .

    """
    type_list=[]
    if type == "Fraud":
        type_list = [1 for _ in range(no_of_records)]
    elif type == "No Fraud":
        type_list = [0 for _ in range(no_of_records)]
    else :
        prob_of_1 = int(percentage[:-1])/100
        type_list = np.random.choice([0,1],size=no_of_records,p=[1-prob_of_1,prob_of_1])
    conditions = pd.DataFrame({'Label': type_list})
    synthesizer = CTGANSynthesizer.load('/content/my_ctgan_model (1).pkl')
    synthetic_data = synthesizer.sample_remaining_columns(conditions)
    return synthetic_data

In [8]:
#generating the data and storing the data in the data variabel
generated_synthetic_data = synthetic_data_gen(no_of_records=100000,type="Both",percentage="50%")
generated_synthetic_data.head()

Sampling remaining columns:  50%|█████     | 50210/100000 [50:07<49:42, 16.69it/s]  
/usr/local/lib/python3.10/dist-packages/sdv/single_table/utils.py:160: UserWarning: Only able to sample 50210 rows for the given conditions. To sample more rows, try increasing `max_tries_per_batch` (currently: 100). Note that increasing this value will also increase the sampling time.
  warnings.warn(user_msg)


,Time_step,Transaction_Id,Sender_Id,Sender_Account,Sender_Country,Sender_Sector,Sender_lob,Bene_Id,Bene_Account,Bene_Country,USD_amount,Label,Transaction_Type
0.0,2022-03-15 10:24:00,sdv-id-0,NaN,ACCOUNT-7915,FALKLAND-ISLANDS,24930.0,NaN,sdv-pii-irrej,sdv-pii-9s8ci,USA,270.04,0,WITHDRAWAL
5.0,2022-03-19 16:32:01,sdv-id-1,JPMC-CLIENT-10098,ACCOUNT-6424,FALKLAND-ISLANDS,NaN,NaN,NaN,sdv-pii-f0moc,NAMIBIA,20.80,0,DEPOSIT-CASH
6.0,2022-03-15 10:24:00,sdv-id-2,JPMC-CLIENT-3719,ACCOUNT-3740,FALKLAND-ISLANDS,22114.0,CCB,NaN,sdv-pii-xl95e,NaN,648.98,0,MOVE-FUNDS
7.0,2022-03-21 05:50:43,sdv-id-3,NaN,ACCOUNT-3728,USA,35171.0,CCB,sdv-pii-6p0hx,sdv-pii-djw62,USA,0.56,0,MOVE-FUNDS
10.0,2022-03-15 10:24:00,sdv-id-4,JPMC-CLIENT-10098,ACCOUNT-823,USA,8034.0,CCB,sdv-pii-zqie5,sdv-pii-f3n24,USA,998.52,0,QUICK-PAYMENT


In [ ]:
generated_synthetic_data.to_csv("generated_data_new_ctgan_model_test.csv")

In [ ]:
def upload_data_to_cloud():
    connection_string = "DefaultEndpointsProtocol=https;AccountName=frauddataproject;AccountKey=X9yDy+C2CyV6oz7bmUYiyCROovcdxrSAVVHq+7GfGMitZhT7ImajRnohuHfkhosvxEFfYYaifqvR+AStb0GJBw==;EndpointSuffix=core.windows.net"
    #container_name = "real-time-data"

    pd_data = pd.DataFrame(generated_synthetic_data)
    if pd_data.shape[0] > 0:
        try:
            now = datetime.now()
            file_path = f"generate_synthetic_data_file_{now.year}{now.month}{now.day}{now.hour}{now.minute}_{now.second}.csv"
            pd_data.to_csv(file_path)
            """
            # Create a BlobServiceClient object
            blob_service_client = BlobServiceClient.from_connection_string(connection_string)

            # Create a client to interact with the container
            container_client = blob_service_client.get_container_client(container_name)

            # Upload the file to the blob

            with open(file_path, "rb") as data:
                blob_client = container_client.upload_blob(name=file_path, data=data)
            print("File Uploaded Successfully")  # Display success message in Streamlit UI
            """
        except Exception as e:
            print(f"Error uploading file: {e}")
    else:
        print("No data Generated")

upload_data_to_cloud()